In [1]:
import asyncio
import logging
import time
from pathlib import Path

# Pyrit imports
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.prompt_target import OpenAIChatTarget, HTTPTarget
from pyrit.score.evaluator import Evaluator
from pyrit.orchestrator import AHAssistantOrchestrator
from pyrit.orchestrator.multi_turn.red_teaming_orchestrator import RTOSystemPromptPaths
from pyrit.common.text_helper import save_html_report, generate_multi_turn_html_report
from pyrit.prompt_target.http_target.parsers.AHAssistantResponseParser import AHAssistantResponseParser

# ------------------------------------------------------------------------------
# Logging Configuration
# ------------------------------------------------------------------------------
logging.basicConfig(level=logging.WARNING)

# ------------------------------------------------------------------------------
# Constants & Configuration
# ------------------------------------------------------------------------------

TOKEN = ""
BASE_URL = ""

# HTTP request template for starting the conversation
start_chat_request_raw = f"""
POST {BASE_URL}
Content-Type: application/json
X-Authorization: {TOKEN}
Accept: */*

{{
    "message": "{{PROMPT}}",
    "model": "gpt-4o-mini"
}}
"""

# Strategy and evaluation config
strategy_path = RTOSystemPromptPaths.AH_ASSISTANT_CUSTOMER.value
scorer_path = "assets/AH_Evaluators/chat_evaluator.yaml"
report_output_path = "tests/E2E/reports/CustomerAutomation"

# ------------------------------------------------------------------------------
# Objectives for Red Teaming
# ------------------------------------------------------------------------------

objectives = [
    "As a home cook, I want a quick and easy pasta recipe for a busy weeknight. I live gluten free and I have a nut allergy. I like meat"
]

# ------------------------------------------------------------------------------
# Main Async Routine
# ------------------------------------------------------------------------------

async def main():
    # Initialize test results and environment
    results = []
    initialize_pyrit(memory_db_type=IN_MEMORY)

    # Loop over each objective
    for objective in objectives:
        start_time = time.time()

        # Evaluator variables for dynamic scoring context
        scorer_variables = {"objective": objective}

        # Create target for both attack & evaluation
        chat_target = OpenAIChatTarget()

        # HTTP target that sends the user's objective
        http_prompt_target = HTTPTarget(
            http_request=start_chat_request_raw,
            prompt_regex_string="{PROMPT}",
            timeout=60.0,
            callback_function=AHAssistantResponseParser.parse_response
        )

        # Evaluator (scoring assistant responses)
        chat_scorer = Evaluator(
            chat_target=chat_target,
            evaluator_yaml_path=Path(scorer_path),
            additional_evaluator_variables=scorer_variables,
            scorer_type="float_scale"
        )

        # Orchestrator coordinates conversation + scoring
        orchestrator = AHAssistantOrchestrator(
            adversarial_chat=chat_target,
            adversarial_chat_system_prompt_path=strategy_path,
            objective_target=http_prompt_target,
            objective_scorer=chat_scorer,
            verbose=True,
            evaluate_chat=True,
            max_turns=10,
            use_score_as_feedback=True
        )

        # Run the attack
        result = await orchestrator.run_attack_async(objective=objective)
        execution_time = time.time() - start_time

        # Get final conversation report
        report = await result.get_conversation_report_async()
        results.append(report)

    # Save final HTML report
    report_dir = Path(report_output_path).resolve()
    report_dir.mkdir(parents=True, exist_ok=True)

    save_html_report(
        results=results,
        is_chat_evaluation=True,
        report_generator=generate_multi_turn_html_report,
        directory=str(report_dir),
        description=(
            "This report provides an overview of multi-turn customer simulations, "
            "highlighting how the assistant engages in realistic, task-focused conversations. "
            "Each dialogue is evaluated for accuracy, helpfulness, and ability to fulfill customer objectives."
        ),
        execution_time=execution_time
    )

# ------------------------------------------------------------------------------
# Entry Point
# ------------------------------------------------------------------------------
await main()



Created new chat with thread_id: 5be4fa41-324b-4023-b9a1-f65082c76c2e

Turn 1

User: Heb je een snel pastarecept voor een drukke doordeweekse avond?

Assistant: {'text_message': 'Ja, hier is een snel pastarecept voor een drukke doordeweekse avond:\n\n**Snel Pasta met Tomatensaus en Spinazie**\n\n**Ingrediënten:**\n- 250 g pasta (bijv. spaghetti of penne)\n- 1 blik tomatenblokjes (400 g)\n- 150 g verse spinazie\n- 2 teentjes knoflook, fijngehakt\n- 2 eetlepels olijfolie\n- Zout en peper naar smaak\n- Geraspte Parmezaanse kaas (optioneel)\n\n**Bereidingswijze:**\n1. Kook de pasta volgens de aanwijzingen op de verpakking.\n2. Verhit ondertussen de olijfolie in een pan op middelhoog vuur. Voeg de knoflook toe en bak deze 1 minuut.\n3. Voeg de tomatenblokjes toe en laat het 5-7 minuten sudderen. Breng op smaak met zout en peper.\n4. Voeg de spinazie toe en laat deze slinken.\n5. Meng de gekookte pasta door de saus en verwarm alles samen nog even kort.\n6. Serveer met geraspte Parmezaanse k